In [1]:
!wget https://spamassassin.apache.org/old/publiccorpus/20050311_spam_2.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2

--2023-02-25 09:04:52--  https://spamassassin.apache.org/old/publiccorpus/20050311_spam_2.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to spamassassin.apache.org (spamassassin.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2059029 (2.0M) [application/x-bzip2]
Saving to: ‘20050311_spam_2.tar.bz2’

20050311_spam_2.tar 100%[===================>]   1.96M  --.-KB/s    in 0.04s   

2023-02-25 09:04:53 (46.6 MB/s) - ‘20050311_spam_2.tar.bz2’ saved [2059029/2059029]

--2023-02-25 09:04:54--  https://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to spamassassin.apache.org (spamassassin.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1029898 (1006K) [application/x-bzip2]
Saving to: ‘20030228_hard_ham.tar

In [2]:
!tar -xf /content/20050311_spam_2.tar.bz2
!tar -xf /content/20030228_hard_ham.tar.bz2
!tar -xf /content/20030228_easy_ham_2.tar.bz2

In [3]:
import os
import pandas as pd
import numpy as np

In [4]:
spam_corpus = []
ham_corpus = []

for filename in os.scandir("/content/easy_ham_2"):
  with open(filename.path, encoding='latin-1') as f:
      ham_corpus.append(f.read())


for filename in os.scandir("/content/hard_ham"):
  with open(filename.path, encoding='latin-1') as f:
      ham_corpus.append(f.read())

for filename in os.scandir("/content/spam_2"):
  with open(filename.path, encoding='latin-1') as f:
      spam_corpus.append(f.read()) 

print(len(spam_corpus), len(ham_corpus))

1397 1652


In [5]:
spam_corpus = np.array(spam_corpus)
ham_corpus = np.array(ham_corpus)

spam_corpus = np.stack((spam_corpus, np.ones(spam_corpus.shape)), axis=-1)
ham_corpus = np.stack((ham_corpus, np.zeros(ham_corpus.shape)), axis=-1)

In [6]:
spam_df = pd.DataFrame(spam_corpus)
ham_df = pd.DataFrame(ham_corpus)
df = pd.concat([spam_df, ham_df])
df.reset_index(drop=True, inplace=True)
df.columns = ["email", "spam"]
df.head()

,email,spam
0,From bestbuys@hotmail.com Sat Jul 20 02:19:58...,1.0
1,From xe8xekv2v2683@hotmail.com Wed Jul 24 13:...,1.0
2,From ilug-admin@linux.ie Tue Jul 30 10:52:20 ...,1.0
3,From mdbayler@amison69.freeserve.co.uk Mon Ju...,1.0
4,From fork-admin@xent.com Thu Jul 25 11:10:15 ...,1.0


In [7]:
del spam_corpus, ham_corpus, spam_df, ham_df

In [8]:
X = df["email"]
y = df["spam"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
y_train.value_counts(normalize=True), y_test.value_counts(normalize=True)

(0.0    0.541615
 1.0    0.458385
 Name: spam, dtype: float64, 0.0    0.542623
 1.0    0.457377
 Name: spam, dtype: float64)

In [ ]:
import re
def clean_email_structure(email: str):
  email = re.sub(r'(<[^<>@]+>)','',email)
  return email

print(clean_email_structure(X[0]))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word', stop_words = "english")
vectorizer.fixed_vocabulary_ = True
X_prep = vectorizer.fit_transform(X_train)
print(len(vectorizer.get_feature_names_out()))
print(X_prep.toarray())

102593
[[4 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [3 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [21]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_prep[:200], y_train[:200])

NameError: ignored

In [ ]:
sgd_clf.predict(vectorizer.transform([X_test.iloc[0]]))

array(['1.0'], dtype='<U3')

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_prep, y_train, cv=3, scoring="accuracy")

array([0.95448954, 0.96432964, 0.95694957])

In [22]:
!pip install urlextract

In [23]:
import urlextract # may require an Internet connection to download root domain
                  # names

url_extractor = urlextract.URLExtract()
some_text = "Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s"
print(url_extractor.find_urls(some_text))

['github.com', 'https://youtu.be/7Pq-S557XQU?t=3m32s']


In [43]:
import re

from sklearn.base import BaseEstimator, TransformerMixin
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

tmp_test_corpus = ["This is a text document?",
                   "There are only 4 rows in this document.", 
                   "This doc. contains: words, symbols and numbers.",
                   "There is a number set: {123, 123.109, 90, 1} https://github.com/geongeorge/i-hate-regex https://www.myfunnyurl.fun/index/about/smt",
                   "Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s"]

class VectorCounterWords(BaseEstimator, TransformerMixin):

  def __init__(self, analyzer: str = "word", split: str = " ", 
               remove_punctuation: bool = True, stem: bool = True,
               replace_nums: bool = True, replace_urls: bool = True,
               is_lowercase: bool = True, remove_css: bool = True):
    
    self.analyzer = analyzer
    self.in_features_ = None
    self.split = split
    self.remove_punctuation = remove_punctuation
    self.replace_nums = replace_nums
    self.replace_urls = replace_urls
    self.is_lowercase = is_lowercase
    self.fixed_vocabulary_ = True
    self.remove_css = remove_css
    self.punctuation_corpus = ['.,!""'"?"]
    self.stem = stem
    self.ps = PorterStemmer()

  def fit(self, X, y=None):
    return self

  def __preprocess_sentence(self, sentence):
    if self.remove_css:
      sentence = clean_email_structure(sentence)
    if self.is_lowercase:
      sentence = sentence.lower()

    if self.replace_urls:
      urls = list(set(url_extractor.find_urls(sentence)))
      urls.sort(key=lambda url: len(url), reverse=True)
      for url in urls:
        sentence = sentence.replace(url, " URL ")

    if self.remove_punctuation:
      sentence = re.sub(r'[^\w\s]','',sentence)

    if self.replace_nums:
      sentence = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', 'NUMBER', sentence)

    sentence= sentence.split(self.split)

    if self.stem:
      sentence = list(map(lambda word: self.ps.stem(word), sentence)) 

    return sentence

  def transform(self, X, y=None):
    X = np.array(X)
    self.in_features_ = []
    X_transformed = []
    for x in X:
      x = self.__preprocess_sentence(x)
      word_counts = Counter(x)
      X_transformed.append(word_counts)
      for feature in word_counts.keys():
        if feature not in self.in_features_:
          self.in_features_.append(feature) 

    count_matrix = np.zeros((len(X_transformed), len(self.in_features_)))
    
    for k, _dict in enumerate(X_transformed):
      for i, feature in enumerate(self.in_features_):
        count_matrix[k, i] = _dict[feature]

    return X

  def get_feature_names_out(self, names=None):
    return self.in_features_


counter = VectorCounterWords()
counter.fit_transform(X_train)

KeyboardInterrupt: ignored

In [44]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer

vectorizer = CountVectorizer(analyzer='word', stop_words = "english")
vectorizer.fixed_vocabulary_ = True

words_pipeline = make_pipeline(
    vectorizer
)

preprocessing = ColumnTransformer(
    transformers=[
    ("words", words_pipeline, "email")
])

pred_pipeline = Pipeline(steps=[
    ("preproc", preprocessing),
    ("pred", SGDClassifier(random_state=42))
])

pred_pipeline.fit(pd.DataFrame(X_train), y_train)

y_pred = pred_pipeline.predict(pd.DataFrame(X_test))

In [45]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        # "preproc__words__vectorcounterwords__remove_punctuation": [True, False],
        # "preproc__words__vectorcounterwords__remove_punctuation": [True, False],
     
        "pred__loss": ["hinge", "log_loss", "log", "modified_huber", "squared_hinge", "perceptron", "squared_error", 
                       "huber", "epsilon_insensitive", "squared_epsilon_insensitive"]
    }
]
grid_search = GridSearchCV(pred_pipeline, param_grid, cv=3, verbose=2,n_jobs=-1)
grid_search.fit(pd.DataFrame(X_train), y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


KeyboardInterrupt: ignored

In [ ]:
best_clf = grid_search.best_estimator_

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(best_clf, pd.DataFrame(X_train), y_train, cv=3, scoring="accuracy")

array([0.96924969, 0.96063961, 0.95325953])

In [ ]:
y_pred = best_clf.predict(pd.DataFrame(X_test))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[318,  13],
       [  9, 270]])

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_test, y_pred, average="binary", pos_label="1.0") , recall_score(y_test, y_pred,  average="binary", pos_label="1.0")

(0.9540636042402827, 0.967741935483871)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average="binary", pos_label="1.0")

0.9608540925266903